# Algorithms for Data Science

## Filtering Stream Items

### 1. Preliminaries 

The objective of this lab is to implement algorithms for filtering "good" items on streams. We will start by the simple implementation using only one hash function, and then it will be required of you to implement the full Bloom filter. We assume a random stream $S$ of $m$ email strings. We assume that the first $g$ emails are the good ones, that we have $n$ bits allocated in the bit array $B$ (for simplicity, implemented as an array here).

In [1]:
import random
from string import ascii_lowercase

#parameters
m = 100
g = 10
stream_size = 10000
n = 512

#generate some random strings of size 5 + 1 + 5
D = []
for _ in range(m):
  D.append(''.join(random.choice(ascii_lowercase) for i in range(5))+\
           '@'+''.join(random.choice(ascii_lowercase) for i in range(5)))

print(D)

['azkxi@oaqga', 'aqlji@yzyxu', 'hacch@irrml', 'geyrj@pngnt', 'hewod@qvipd', 'aakdk@footu', 'kkovw@khpzj', 'ckrfy@qrirg', 'rwyak@pmdpc', 'buhso@htjgc', 'tvros@kivrb', 'snfmr@txdvp', 'bvkqe@rdckp', 'vgowp@cagjv', 'rxpeo@pwwte', 'dmneo@cxydd', 'vtlec@fvpaj', 'egfqx@bpflc', 'volvp@erisq', 'kxcvn@digjk', 'warom@bbzco', 'jnzni@zqaqe', 'cxipd@hwrky', 'vjykv@yretw', 'hlxhw@unvoi', 'ypouh@jgzrg', 'kyvum@zmmpq', 'njfnr@jfqcc', 'pqpdt@otfkz', 'lmsca@vfvkx', 'icsba@xmdup', 'julud@bnhfd', 'zyvlx@wdlhm', 'iihtj@rljuz', 'cjibv@tzyer', 'xavpo@tlhfa', 'jzhrv@atxms', 'rmktg@yoneb', 'husvu@vbkjx', 'uonxo@ioezx', 'whpte@gxtoy', 'saabb@hgkjp', 'ubmtf@zvuzz', 'movkq@sksjd', 'iujdw@ywzho', 'vzase@lwvsh', 'hhpqv@prmws', 'ssefn@ivmgy', 'jaulj@mpudd', 'kwanf@xtcpi', 'vlrmq@tbwka', 'fzuci@aezea', 'myioh@mfgqm', 'hfspu@iikgm', 'myefo@nsgmc', 'vrdjv@ksotf', 'udawh@emacs', 'blmtk@idnwq', 'dkpnm@ewbmf', 'oqkab@efaxr', 'nihcs@xlqfm', 'qjwtj@dfsds', 'jiydq@ljvwh', 'fodrq@tbglg', 'ysyuk@xhrnl', 'zwxgo@tajmf', 'hulkr@gh

### 2. Creating a Hash Function, Filtering Items Using a Single Hash

In the following we create a hash function $h(x)$, which also takes as a parameter a value and $n$, and returns a value in $0\dots n-1$. We populate the byte array $B$, and then we simulate a stream taking random values from $D$ and checking whether the value is good or not. We measure the true positive, false positive, and false negative rates. 

In [2]:
n = 128

#hash function
def h(x,n):
  return hash(x)%n

good_set = set(D[:g]) #just for checking TP and FP rates

#allocate the array of 0s
B = [0] * n

#fill the byte array
for i in range(g): B[h(D[i],n)] = 1
    
print(B)

tp = 0 # good items passing
fp = 0 # bad items passing
tn = 0 # bad items discarded
fn = 0 # good items discarded

#simulate a stream
for _ in range(stream_size):
  #take a random email
  s = random.choice(D)
  #check its hash value
  if B[h(s,n)]==1: #good
    if s not in good_set:
      fp += 1
    else:
      tp += 1
  else: #bad 
    if s in good_set:
      fn += 1
    else:
      tn += 1

print('False positive rate: %f'%(float(fp)/float(tn+fp)))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
False positive rate: 0.089446


We may want to create a random hash function that can also be pairwise independent when we will need to generate $k$ independent pairwise hashes.
The following procedure can be implemented:
* choose a large prime number $p$
* generate two random numbers $a$ and $b$ in the range $\{1,\dots,p\}$
* the hash is then $h_{a,b}(x)=ax+b \mod p$
* we can also restrict it into $\{0,\dots,n-1\}$

In [3]:
p = 1223543677

a = random.randrange(p)
b = random.randrange(p)

def h(x,a,b,p,n):
  return ((a*hash(x)+b)%p)%n
#remark: here we use hash(x) instead of the values to allow for all hashable python types
#   e.g., strings, tuples

#reinitialize the array, for testing
B = [0] * n

for i in range(g): 
  B[h(D[i],a,b,p,n)] = 1

print(B)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### 3. **TASK** - Bloom Filters

Your task is to implement the Bloom filters as described in the class lecture. For this, you have to:
1. generate $k$ random pairwise independent hash functions (_hint_: use the example shown above)
2. initialize $B$, by setting $1$ in each $h_i(x)$, $i\in\{1,\dots,k\}$, for all items $x$ in the good set
3. an item $s$ in the stream is considered good if, for all $i\in\{1,\dots,k\}$, we have $B[h_i(s)]=1$

Measure the true positive and false positive rate for various values of $k$ and compare to the values obtained when setting $k=n/m\ln 2$ (to the nearest integer value). What do you notice?

Rates:

$
  \text{false positive rate} \frac{FP}{FP+TN}
$

$
  \text{true positive rate} \frac{TP}{TP+FN}
$

In [5]:
# YOUR CODE HERE
n = 128

good_set = set(D[:g]) #just for checking TP and FP rates

p = 1223543677

ab_pairs = list()
k = 2

def generate_ab_pair():
    for i in range(k):
        a = random.randrange(p)
        b = random.randrange(p)
        ab_pairs.append((a,b))

generate_ab_pair()

def h(x,a,b,p,n):
  return ((a*hash(x)+b)%p)%n
#remark: here we use hash(x) instead of the values to allow for all hashable python types
#   e.g., strings, tuples


#allocate the array of 0s
B = [0] * n

#fill the byte array
for i in range(g): 
    for a,b in ab_pairs:
        B[h(D[i],a,b, p, n)] = 1
    
print(B)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0]


In [6]:
tp = 0 # good items passing
fp = 0 # bad items passing
tn = 0 # bad items discarded
fn = 0 # good items discarded

#simulate a stream
for _ in range(stream_size):
    #take a random email
    s = random.choice(D)
    #check its hash value
    
    count = 0
    for a,b in ab_pairs:
        if B[h(s,a,b,p,n)]==1: #good
            count += 1
    if count == k:
        if s not in good_set:
          fp += 1
        else:
          tp += 1
    else: #bad 
        if s in good_set:
          fn += 1
        else:
          tn += 1

print('False positive rate: %f'%(float(fp)/float(tn+fp)))

False positive rate: 0.032620


_You can use this cell to write your discussion of the results_